In [ ]:
#Calculate average accuracy    
    def calculate_acc(self,iterate_times):
        
        self.mass        = 0
        self.origin_mass = 0
        self.closest     = 0
        self.origin_label_table = np.arange(self.K)
        self.acc         = 0
        
        i = 0
        
        while( i < iterate_times):
            self.run()
            temp_acc = self.cal_1_acc()
            
            # Avoid the rare situations that some cluster are gone
            if len(np.unique(self.closest)) != self.K:
                continue
            
            i += 1
            self.acc += temp_acc
            
        self.acc /= i
        return self.acc

    
#---------------------------------------------------------------------------------
#----------------Subfunctions of calculate_acc(iterate_times)---------------------
#---------------------------------------------------------------------------------
    #all the clusters should be 1-D DataFrame which contains the same labels
    def find_mass(self,labels):
        mass      = np.zeros((self.K,self.DIM))
        num       = np.zeros(self.K)
        row_count = 0

        for i in labels.iloc[0:,0].values:
            for j in range(self.DIM):
                mass[i][j] += self.data_x.iloc[row_count][j]
            row_count += 1
            num[i] += 1

        for i in range(self.K):
            for j in range(self.DIM):
                mass[i][j] /= num[i]

        return mass   

    def calculate_closest(self):
        closest = np.zeros(self.K)
        for i in range(self.K):
            min_dist=float("inf")
            for j in range(self.K):
                dist = np.linalg.norm(self.mass[i]-self.origin_mass[j])
                if dist < min_dist:
                    min_dist = dist
                    closest[i]=j
        return closest 
    
    def relabel(self,origin_table,rename_table,target):
        target = target.replace(origin_table,rename_table)
        return target
    
    
    def cal_1_acc(self):    
        self.mass        = self.find_mass(self.table)
        self.origin_mass = self.find_mass(self.label)
        self.closest     = self.calculate_closest()
        self.table       = self.relabel(self.origin_label_table,self.closest,self.table)
                    
        return accuracy_score(self.label,self.table)
    
    